In [83]:
a = !pip install simpletransformers
print('Done')

Done


In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
from google.colab import drive
import re
from sklearn.model_selection import train_test_split
from simpletransformers.language_representation import RepresentationModel
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [2]:
drive.mount('/content/drive')
text = []
with open('/content/drive/MyDrive/Clinical Semantic Textual Similarity/clinicalSTS2019/train/clinicalSTS2019.train.txt', 'r') as f:
    for line in f.readlines():
      line = line.replace(' ', '"')
      line = line.replace('"', ' ')
      line = line.replace('\n', ' ')
      line = re.split('\t', line)
      text.append(line)

Mounted at /content/drive


In [4]:
model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=False
    )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
text_embedded=[]
label=[]
dataset = []

for i in range(len(text)):
  sentence_vectors = model.encode_sentences(text[i][0:2], combine_strategy="mean")
  text_embedded.append(sentence_vectors)
  label.append(float(text[i][2]))

for i in range(len(text_embedded)):
  data = np.concatenate((text_embedded[i][0], text_embedded[i][1]), axis=0, out=None)
  dataset.append(data)

X_train, X_test, y_train, y_test = train_test_split(dataset, label, random_state=104, test_size=0.25, shuffle=True)

y_train = np.array(y_train)
y_test = np.array(y_test)

X_train = np.array(X_train)
X_test = np.array(X_test)

In [84]:
def baseline_model():
# create model
  model = Sequential()
  model.add(Dense(1536, input_shape=(1536,), kernel_initializer='normal', activation='sigmoid'))
  model.add(Dense(1000, activation='relu'))
  model.add(Dense(500, activation='relu'))
  model.add(Dense(250, activation='relu'))  
  model.add(Dense(25, activation='relu'))  
  model.add(Dense(1, kernel_initializer='normal'))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=1)

<ipython-input-84-562045f76b4d>:14: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=1)


In [85]:
estimator.fit(X_train, y_train)

Epoch 1/50
247/247 [==============================] - 12s 48ms/step - loss: 1.7796
Epoch 2/50
247/247 [==============================] - 12s 47ms/step - loss: 1.2446
Epoch 3/50
247/247 [==============================] - 12s 48ms/step - loss: 1.2012
Epoch 4/50
247/247 [==============================] - 11s 46ms/step - loss: 1.0487
Epoch 5/50
247/247 [==============================] - 11s 45ms/step - loss: 0.9327
Epoch 6/50
247/247 [==============================] - 12s 50ms/step - loss: 0.8398
Epoch 7/50
247/247 [==============================] - 12s 47ms/step - loss: 0.7214
Epoch 8/50
247/247 [==============================] - 11s 46ms/step - loss: 0.6276
Epoch 9/50
247/247 [==============================] - 11s 45ms/step - loss: 0.5593
Epoch 10/50
247/247 [==============================] - 11s 45ms/step - loss: 0.4761
Epoch 11/50
247/247 [==============================] - 13s 52ms/step - loss: 0.4269
Epoch 12/50
247/247 [==============================] - 11s 46ms/step - loss: 0.3806
E

In [86]:
y_pred = estimator.predict(X_test)
y_pred[0:15]

83/83 [==============================] - 1s 15ms/step


array([3.5773168, 1.2410508, 2.761137 , 2.7151198, 2.186749 , 3.8808203,
       4.093165 , 1.5626189, 2.3120804, 4.6806474, 3.413715 , 1.4806215,
       1.4575315, 0.8821032, 4.4781113], dtype=float32)

In [80]:
y_test[0:15]

array([4.9 , 1.  , 2.  , 0.5 , 1.5 , 4.  , 4.25, 0.5 , 2.25, 4.4 , 2.5 ,
       1.65, 1.  , 3.  , 3.5 ])

In [81]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print(mean_squared_error(y_test, y_pred),mean_absolute_error(y_test, y_pred), r2_score(y_test, y_pred))

1.258116636045453 0.8505762865907375 0.38926964845129175


prestazioni da migliorare assoutamente, o modificando l'architettura di rete o idealmente cambiando approccio da principio: attualemente ogni X_train è il composto dell embedding delle due frasi concatenate.
si deve esplorare la possibilita di non concatenare le frasi e passarle alla rete descritta di sotto, come input separati da unire all' interno della rete 

In [89]:
from keras.engine import input_layer
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
import keras

net= Sequential()
net.add(keras.Input(shape =(2, 768)))
net.add(Dense(256))
net.add(Dense(128))
net.add(Dense(64))
net.add(Flatten())
net.add(Dense(64))
net.add(Dense(16))
net.add(Dense(4))
net.add(Dense(1))
net.add(Activation('softmax'))
net.compile(loss='mean_absolute_error', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

net.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 2, 256)            196864    
                                                                 
 dense_32 (Dense)            (None, 2, 128)            32896     
                                                                 
 dense_33 (Dense)            (None, 2, 64)             8256      
                                                                 
 flatten_6 (Flatten)         (None, 128)               0         
                                                                 
 dense_34 (Dense)            (None, 64)                8256      
                                                                 
 dense_35 (Dense)            (None, 16)                1040      
                                                                 
 dense_36 (Dense)            (None, 4)                